In [184]:
from collections import Counter
from itertools import repeat
import json
import pickle
import warnings


from hyperopt import fmin, hp, tpe, STATUS_OK, Trials
import pandas as pd
import numpy as np
import random

np.random.seed(42)
random.seed(42)

warnings.filterwarnings('ignore')

In [185]:
# x_dynamic = pd.read_csv("./processed_data/dynamic_feats.csv")
game_logs = pd.read_csv("./processed_data/processed_game_logs.csv")
mn_twins_mask = game_logs["HomeTeam"]=="MIN"
season_mask = game_logs["year"]==2022
prev_yr_games = game_logs[season_mask].reset_index(drop=True)

## RULES
- The 81 home games must be scheduled on dates between April 1, 2023 and September 30, 2023.
- A maximum of 41 weekend game dates (Fri, Sat, or Sun) are allowed.
- The optimal schedule must include the same quantity of games against the opponents listed on the current 2023 MN Twins’ home game schedule. You may re-allocate these opponents across the season as you see fit.
- To minimize travel for the teams, each MN Twins opponent must be scheduled for a minimum of two consecutive games before a new opponent can be played.

In [186]:
# Teams coming to MN
## Current Schedule
visitor_sequence = "HOU"*3+"CHA"*3+"WAS"*3+"NYA"*3+"KCA"*4+"SDN"*3+"CHN"*3+"SFN"*3+"TOR"*3+"CLE"*4+"MIL"*4+"DET"*4+"BOS"*4+"KCA"*3+"BAL"*3+\
                "CHA"*3+"SEA"*3+"ARI"*3+"DET"*2+"PIT"*3+"TEX"*4+"CLE"*3+"NYN"*2+"TBA"*3+"ANA"*3+"OAK"*3

visitor_sequence = [visitor_sequence[i:i+3] for i in range(0, len(visitor_sequence), 3)]
visitor_counts = Counter(visitor_sequence)

visitor_sets = {}
for team, num_matches in visitor_counts.items():
    if num_matches in [2,3]:
        # Only one set of possibilities
        visitor_sets[team] = [list(repeat(team, num_matches))]
    elif num_matches ==4:
        # series of 2-2 or 4
        visitor_sets[team] = [[list(repeat(team, 4))], 
                              [list(repeat(team, 2)), list(repeat(team, 2))]]
    elif num_matches ==6:
        # series of 6, 4-2 or 2-2-2
        visitor_sets[team] = [[list(repeat(team, 6))],
                              [list(repeat(team, 4)), list(repeat(team, 2))],
                              [list(repeat(team, 3)), list(repeat(team, 3))],
                              [list(repeat(team, 2)), list(repeat(team, 2)), list(repeat(team, 2))]]
    elif num_matches ==7:
        # series of 7, 4-3 or 2-2-3
        visitor_sets[team] = [[list(repeat(team, 7))],
                              [list(repeat(team, 4)), list(repeat(team, 3))],
                              [list(repeat(team, 3)), list(repeat(team, 2)), list(repeat(team, 2))]]
        # Recursive logic
        # remaining_matches = num_matches
        # visitor_sets[team] = []
        # while remaining_matches>3:
        #     remaining_matches = num_matches-3
        #     visitor_sets[team].append(list(repeat(team, remaining_matches)))

In [187]:
NUM_WEEKEND_GAMEDAYS = 41
NUM_GAMES = len(visitor_sequence)
NUM_SIMULATIONS = 10000
MAX_TRIALS = 100

In [188]:
# Generating date range
date_space = pd.to_datetime(pd.date_range(start="2023-04-01", end="2023-09-30"))
date_df = pd.DataFrame(index=range(len(date_space)))
date_df["year"] = date_space.year
date_df["DayofMonth"] = date_space.day
date_df["Month"] = date_space.month
date_df["DayofWeek"] = date_space.day_of_week

# selecting weekend dates
weekend_mask = date_df["DayofWeek"].isin([4,5,6])

In [189]:
param_space = {}
DAYFLAG_SAMPLES = [np.random.randint(size=82, low=0, high=2) for i in range(NUM_SIMULATIONS)]
param_space["DayFlag"] = list(range(NUM_SIMULATIONS))

WEEKEND_SAMPLES = [date_df[weekend_mask].sample(NUM_WEEKEND_GAMEDAYS) for i in range(NUM_SIMULATIONS)]
param_space["weekend_sample"] = list(range(NUM_SIMULATIONS))

WEEKDAY_SAMPLES = [date_df[~weekend_mask].sample(NUM_GAMES-NUM_WEEKEND_GAMEDAYS) for i in range(NUM_SIMULATIONS)]
param_space["weekday_sample"] = list(range(NUM_SIMULATIONS))

VISTING_TEAM_SAMPLES = []
unique_team_combos = sum([len(x) for x in visitor_sets.values()])
for idx in range(unique_team_combos):
    team_schedule = []
    for k,v in visitor_sets.items():
        if len(v)>1:
            choice_idx = np.random.randint(low=0, high=len(v))
            team_schedule+=v[choice_idx]
        else:
            team_schedule+=v
    VISTING_TEAM_SAMPLES.append(np.array([i for team_set in team_schedule for i in team_set]))

param_space["visiting_team"] = list(range(unique_team_combos))

In [190]:
with open("./model/lgb_static_model.pkl","rb") as f:
    static_model = pickle.load(f)

In [191]:
ranking_info_cols = ["total_wins","day_league_rank","win_rate"]

group_cols = ["HomeTeam","VisitingTeam"]

other_league_cols = ['nba_season_flag',
       'nhl_season_flag']

common_features = [ "HomeTeamGameNumber",
                    "VisitingTeamGameNumber",
                    # Calender features
                    "year",
                    "DayofWeek",
                    "Month",
                    "DayofMonth",
                    "DayFlag",
                    # Stadium Features
                    "park_age",
                    "NonRegular_ParkFlag",
                    "StadiumCapacity"]+\
                    ["home_payroll", "visiting_payroll","home_top_salary", "visiting_top_salary"] +\
                    [ # Miscellaneous time based cols
                    "same_opp", "days_since_last_match","first_home_game", 'league_rival_col'] +\
                    other_league_cols #'covid_effect']

prev_season_features = ["HomeAttendance_last_year",
                        "VisitingAttendance_last_year",
                        "Attendance_last_year",
                        "Homewin_rate_last_year",
                        "Homeday_league_rank_last_year",
                        "HomeTeamOffense_Homeruns_last_year",
                        "HomeTeamOffense_Strickouts_last_year",
                        "HomeTeamPitchers_TeamEarnedRuns_last_year",
                        "Visitingwin_rate_last_year",
                        "Visitingday_league_rank_last_year",
                        "VisitingTeamOffense_Homeruns_last_year",
                        "VisitingTeamOffense_Strickouts_last_year",
                        "VisitingTeamPitchers_TeamEarnedRuns_last_year",
                        "HomeTeamScore_last_year",
                        "VistingTeamScore_last_year"]

In [192]:
def create_season_date_index(start, end, league_name):
    season_date_index = [day
        for year in range(2000,2023)
        for day in pd.date_range(
            start=start+str(year), end=end+str(year+1), freq="D")]
    league_ind_df = pd.DataFrame(index=pd.to_datetime(season_date_index))
    league_ind_df[league_name+"_season_flag"] = 1
    return league_ind_df


def edit_col_name(col_list=ranking_info_cols,
                  prefix=None,
                  suffix=None):
    if suffix:
        col_list = {x: x+suffix for x in col_list}
    if prefix:
        col_list = {x: prefix+x for x in col_list}
    return col_list

# Creating 2023 season features
def add_feats(df, prev_yr_df):
    df["league_rival_col"] = (df["VisitingTeamLeague"]==df["HomeTeamLeague"]).astype(int)
    df["same_opp"] = (df["VisitingTeam"] == df.groupby(["HomeTeam","year"])["VisitingTeam"].shift(1))
    df["same_opp"] = df.groupby(["HomeTeam","year","VisitingTeam"])["same_opp"].cumsum()

    df["days_since_last_match"] = (df.groupby(["HomeTeam","year"])["Date"].shift(1) - df["Date"]).dt.days.fillna(0)

    df["first_home_game"] = (df["Date"]==df.groupby(["HomeTeam","year"])["Date"].transform("first")).astype(int)

    nba_ind_df = create_season_date_index(start="15-10-", end="15-06-", league_name="nba")
    nhl_ind_df = create_season_date_index(start="15-10-", end="30-06-", league_name="nhl")
    nfl_ind_df = create_season_date_index(start="10-09-", end="15-02-", league_name="nfl")

    other_league_cols = []
    for league_df in [nba_ind_df,
                    nhl_ind_df,
                    nfl_ind_df]:
        df = df.merge(league_df, left_on="Date", right_index=True, how="left")
        league_col = league_df.columns[0]
        other_league_cols.append(league_col)
        df[league_col] = df[league_col].fillna(0)

    alt_sports = pd.read_csv("./processed_data/other_leagues.csv")

    with open("./processed_data/code_city_map.json", "r") as f:
        city_code_map = json.load(f)

    df["City"] = df["HomeTeam"].map(city_code_map)

    df = (df.merge(alt_sports,
                on="City",
                how="left"))

    alt_sports_cols = ["NBA","NHL","NFL"]
    for league, ind_col in zip(alt_sports_cols, other_league_cols):
        df[ind_col] = df[league].astype(int)*df[ind_col]

    df.drop(columns=alt_sports, inplace=True)

    for group_col in ["Home","Visiting"]:
        agg_dict = {"Attendance": "mean",
                    group_col+"win_rate": "last",
                    group_col+"day_league_rank": "last",
                    group_col+"TeamOffense_Homeruns": sum,
                    group_col+"TeamOffense_Strickouts": sum,
                    group_col+"TeamPitchers_TeamEarnedRuns": sum}

        # Aggregating season stats and shifting for lag effect
        lagged_df = pd.DataFrame(columns=[group_col+"Team", "year"], index=prev_yr_df.index)
        lagged_df[[group_col+"Team","year"]] = prev_yr_df[[group_col+"Team","year"]]
        stat_cols = list(agg_dict.keys())
        lagged_df[stat_cols] = prev_yr_df.groupby([group_col+"Team","year"], as_index=False)[stat_cols].shift(0)
        lagged_df = lagged_df.groupby([group_col+"Team","year"], as_index=False).agg(agg_dict)
        
        col_rename_map = edit_col_name(stat_cols, suffix="_last_year")
        col_rename_map["Attendance"] = group_col+"Attendance_last_year"
        lagged_df.rename(columns=col_rename_map, inplace=True)
        lagged_df = lagged_df.drop_duplicates(subset=[group_col+"Team"])

        # merge the last year"s average attendance into the original dataframe
        df = pd.merge(df,
                    lagged_df.drop(columns="year"),
                    how="left",
                    on=[group_col+"Team"])

    # Lagged and Aggregated featres for Home and Visiting Team
    agg_dict = {"Attendance": "mean",
                "HomeTeamScore": sum,
                "VistingTeamScore": sum}

    # Aggregating season stats and shifting for lag effect
    lagged_df = prev_yr_df.groupby(group_cols+["year"], as_index=False).agg(agg_dict)#.shift(1).fillna(0)
    lagged_df.rename(columns=edit_col_name(col_list=list(agg_dict.keys()), suffix="_last_year"),
                    inplace=True)
    lagged_df = lagged_df.drop_duplicates(subset=["HomeTeam","VisitingTeam"])
    # merge the last years average attendance into the original dataframe
    
    df = pd.merge(df, lagged_df.drop(columns="year"),
                how="left",
                on=group_cols,
                suffixes=("", "_last_year"))
    return df

In [193]:

def run_sim(params):
    test_df = pd.concat([WEEKEND_SAMPLES[params["weekend_sample"]],
                         WEEKDAY_SAMPLES[params["weekday_sample"]]], ignore_index=True
                        ).sort_values(["Month","DayofMonth"]).reset_index(drop=True)

    test_df["VisitingTeamGameNumber"] = prev_yr_games["VisitingTeamGameNumber"]
    test_df["HomeTeamGameNumber"] = prev_yr_games["HomeTeamGameNumber"]
    test_df[["HomeTeamGameNumber","VisitingTeamGameNumber"]] = test_df[["HomeTeamGameNumber","VisitingTeamGameNumber"]].ffill()

    test_df["VisitingTeam"] = VISTING_TEAM_SAMPLES[params["visiting_team"]]

    test_df["HomeTeam"] = "MIN"
    merge_cols = ["park_age", "NonRegular_ParkFlag", "StadiumCapacity"]+\
                ["VisitingTeamLeague","HomeTeamLeague"] +\
                ["home_payroll", "visiting_payroll","home_top_salary", "visiting_top_salary"]
    match_cols = ["HomeTeam","VisitingTeam"]

    test_df = test_df.merge(prev_yr_games[merge_cols+match_cols], on=match_cols, how="left")

    # Stadium age increased by 1
    test_df["park_age"] = test_df["park_age"]+1

    test_df["Date"] = pd.to_datetime(
        test_df["year"].astype(str)+"-"+test_df["Month"].astype(str)+"-"+test_df["DayofMonth"].astype(str))

    test_df = test_df.drop_duplicates(["Date"])

    test_df = add_feats(test_df.copy(), prev_yr_games).reset_index(drop=True)

    test_df["DayFlag"] = [np.random.choice([0,1]) for i in range(len(test_df))]

    pred_attendance = static_model.predict(test_df[common_features+prev_season_features])
    pred_attendance[pred_attendance>test_df["StadiumCapacity"]] = test_df["StadiumCapacity"].values[0]
    return {"loss": -1*pred_attendance.sum(),
                "status": STATUS_OK,
                "predictions": pred_attendance,
                "params": params}

In [195]:
param_grid = {}
for param, space in param_space.items():
        param_grid[param] = hp.choice(param, space)

trials = Trials()
best_params = fmin(fn=run_sim,
        space=param_grid,
        max_evals=1000,
        algo=tpe.suggest,
        verbose=True,
        trials=trials)

100%|██████████| 1000/1000 [30:56<00:00,  1.86s/trial, best loss: -2761157.7584386994]


In [196]:
last_year_attendance = prev_yr_games.loc[mn_twins_mask,"Attendance"].sum()
(-1*min(trials.losses())-last_year_attendance)/last_year_attendance

0.31657019845348766

In [197]:
# get next season schedule
# best_params = trials.best_trial["result"]["params"]
season_schedule = pd.concat([WEEKEND_SAMPLES[best_params["weekend_sample"]],
                         WEEKDAY_SAMPLES[best_params["weekday_sample"]]], ignore_index=True
                        ).sort_values(["Month","DayofMonth"]).reset_index(drop=True)
season_schedule["VisitingTeam"] = VISTING_TEAM_SAMPLES[best_params["visiting_team"]]
season_schedule["DayFlag"] = [np.random.choice([0,1]) for i in range(len(season_schedule))]

In [216]:
(season_schedule[season_schedule["VisitingTeam"].isin(["NYA","SDN","CHA","TEX","BOS","LAN"])]["DayofWeek"].isin([4,5,6])).mean(), (season_schedule["DayofWeek"].isin([4,5,6])).mean()
(season_schedule[~season_schedule["VisitingTeam"].isin(["NYA","SDN","CHA","TEX","BOS","LAN"])]["DayofWeek"].isin([4,5,6])).mean()

0.532258064516129

In [215]:
season_schedule[season_schedule["VisitingTeam"].isin(["NYA","SDN","CHA","TEX","BOS","LAN"])]["DayFlag"].mean(), (season_schedule["DayFlag"]).mean()

(0.3, 0.3780487804878049)

In [211]:
season_schedule.groupby("DayofWeek")["DayFlag"].sum(), season_schedule.groupby("DayofWeek")["DayFlag"].size()

(DayofWeek
 0    8
 1    0
 2    2
 3    3
 4    5
 5    8
 6    5
 Name: DayFlag, dtype: int32,
 DayofWeek
 0    12
 1     9
 2    10
 3    10
 4    14
 5    16
 6    11
 Name: DayFlag, dtype: int64)

In [213]:
(10)/43

0.23255813953488372

In [212]:
(8+8+5)/(12+16+11)

0.5384615384615384

In [207]:
season_schedule["Month"].value_counts()

6    20
8    15
7    13
5    12
4    11
9    11
Name: Month, dtype: int64

In [209]:
35/82

0.4268292682926829